In [1]:
import re
import pandas as pd

In [2]:
df_invalid_histologies_mapping = pd.read_csv('invalid ICDO3 histologies mapping.csv')
df_valid_histologies = pd.read_csv('valid ICDO3 histologies.csv')

In [3]:
df_PathCHART = pd.read_csv('2024_CPC_SMVL_08292023.csv')
df_PathCHART['histology'] = df_PathCHART['Hist'].astype(str)+'/'+df_PathCHART['Behavior'].astype(str)
df_PathCHART['topography'] = df_PathCHART['C_Site'].str[:3]+'.'+df_PathCHART['C_Site'].str[3:4]
df_PathCHART['condition'] = df_PathCHART['histology']+'-'+df_PathCHART['topography']
df_PathCHART_likely = df_PathCHART[df_PathCHART['Status']==1]
df_PathCHART_likely = df_PathCHART_likely.reset_index()
df_PathCHART_likely = df_PathCHART_likely[['condition','histology','topography']]

df_PathCHART_32_likely = df_PathCHART_likely.loc[df_PathCHART_likely['histology'].isin(df_valid_histologies['histology'])]
df_PathCHART_non32_likely = df_PathCHART_likely.loc[~df_PathCHART_likely['histology'].isin(df_valid_histologies['histology'])]
df_t = df_PathCHART_non32_likely.set_index('histology').join(df_invalid_histologies_mapping.set_index('histology_old'),how='inner')
df_t['condition_new']=df_t['histology_new']+'-'+df_t['topography']
df_t.reset_index(drop=True, inplace=True)
df_t.rename({'index':'histology_old'},axis='columns',inplace=True)
df_t.rename({'histology':'histology_old'},axis='columns',inplace=True)
df_t = df_t[['condition_new','histology_new','topography']]
df_t.rename({'condition_new':'condition','histology_new':'histology'},axis='columns',inplace=True)
df_PathCHART_likely = pd.concat([df_PathCHART_32_likely,df_t])
df_PathCHART_likely = df_PathCHART_likely.drop_duplicates()

In [4]:
len(df_PathCHART_likely)

45910

In [5]:
with open('valid_codes.txt', 'r') as f:
    IARC_valid_codes = []
    for line in f:
        IARC_valid_codes.append(line[9:21])
IARC_valid_codes = set(IARC_valid_codes)        
with open('codes.chk', 'r') as f:
    IARC_codes_with_warning = []
    for line in f:
        if re.match(r'^[0-9]+:', line):
            IARC_codes_with_warning.append(line[9:21])
            
IARC_codes_with_warning = set(IARC_codes_with_warning)
IARC_likely_codes = set(IARC_valid_codes.difference(IARC_codes_with_warning))
df_IARC_likely = pd.DataFrame(data=IARC_likely_codes)
df_IARC_likely.rename(mapper={0:'condition'},axis='columns',inplace=True)
df_IARC_likely['histology']=df_IARC_likely['condition'].str[:6]
df_IARC_likely['topography']=df_IARC_likely['condition'].str[7:12]

df_IARC_32 = df_IARC_likely.loc[df_IARC_likely['histology'].isin(df_valid_histologies['histology'])]
df_IARC_non32 = df_IARC_likely.loc[~df_IARC_likely['histology'].isin(df_valid_histologies['histology'])]
df_t = df_IARC_non32.set_index('histology').join(df_invalid_histologies_mapping.set_index('histology_old'),how='inner')
df_t['condition_new']=df_t['histology_new']+'-'+df_t['topography']
df_t.reset_index(drop=True, inplace=True)
df_t.rename({'index':'histology_old'},axis='columns',inplace=True)
df_t.rename({'histology':'histology_old'},axis='columns',inplace=True)
df_t = df_t[['condition_new','histology_new','topography']]
df_t.rename({'condition_new':'condition','histology_new':'histology'},axis='columns',inplace=True)
df_IARC_likely = pd.concat([df_IARC_32,df_t])
df_IARC_likely = df_IARC_likely.drop_duplicates()

In [6]:
len(df_IARC_likely)

63163

In [7]:
df_likely = pd.concat([df_IARC_likely,df_PathCHART_likely])
df_likely = df_likely.drop_duplicates()
df_likely.reset_index(drop=True, inplace=True)

In [8]:
len(df_likely)

74057

In [20]:
df_OMOP = pd.read_csv('icdo3_valid_combination.csv')
df_OMOP['topography'] = df_OMOP['site'].str.strip("'")
df_OMOP['histology'] = df_OMOP['histology_behavior'].str.strip("'")
df_OMOP['condition'] = df_OMOP['histology']+'-'+df_OMOP['topography']
df_OMOP = df_OMOP[['condition','histology','topography']]

In [21]:
df_OMOP

,condition,histology,topography
0,9080/0-C72.4,9080/0,C72.4
1,8052/2-C06.9,8052/2,C06.9
2,9200/0-C31.2,9200/0,C31.2
3,9691/3-C10.2,9691/3,C10.2
4,9727/3-C42.2,9727/3,C42.2
...,...,...,...
59187,9712/3-C49.0,9712/3,C49.0
59188,9702/3-C09.8,9702/3,C09.8
59189,8980/3-C16.6,8980/3,C16.6
59190,9505/1-C71.1,9505/1,C71.1


In [22]:
df_all = pd.concat([df_likely,df_OMOP])
df_all = df_all.drop_duplicates()
df_all.reset_index(drop=True, inplace=True)

In [23]:
len(df_all)

83762

In [24]:
len(df_all) - len(df_OMOP)

24570

In [25]:
df_to_excel['histology_behavior'] = '''''df_all['histology']

= df_all[['histology','topography']]

df_to_excel.rename({'histology':'histology_behavior','topography':'site'},axis='columns',inplace=True)

C:\Temp\ipykernel_28740\3719078262.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_to_excel.rename({'histology':'histology_behavior','topography':'site'},axis='columns',inplace=True)


In [ ]:
df_to_excel.to_csv('icdo3_valid_combination_new.csv',index=False)

In [26]:
df_to_excel

,histology_behavior,site
0,9701/3,C24.0
1,9650/3,C39.8
2,8940/3,C51.0
3,8001/3,C76.4
4,9691/3,C49.2
...,...,...
83757,9754/3,C13.8
83758,9220/0,C31.8
83759,8642/1,C62.9
83760,8120/2,C26.9
